# Pre Work

## Importing libraries

In [78]:
# libraries that deal with grabbing from the .env file
from dotenv import load_dotenv
import os

# make url requests
import requests

# make DFs
import pandas as pd

# show images in notebook
from IPython.display import display, Image

## Getting the API key from the .env file

In [79]:

# Remove the cached environment variable (if any)
os.environ.pop('riot_api_key', None)

# Reload the .env file
load_dotenv()

# Fetch the API key
api_key = os.environ.get('riot_api_key')



# Inputs

## Summoner ID and Region inputs

In [80]:
# input username, region code here, asking for user input

# Prompt user for input
gameName = input("Enter your summoner: ")


# Prompt user for input with a default value
tagLine = input("Enter tagline (default: NA1): ")

# Use default value if input is empty
tagLine = tagLine if tagLine.strip() else "NA1"

# Prompt user for input with a default value
region = input("Enter Region: americas, asia, and europe (default: americas): ")

# Use default value if input is empty
region = region if region.strip() else "americas"

print(gameName, tagLine, region)


hgillis NA1 americas


# Creating Functions

## get puuid from gameName and tagLine

In [81]:
def get_puuid(summonerId=None, gameName=None, tagLine=None, region='americas'):
    """Gets the puuid from a summonerId or Riot_id and riot_tag
    
    Args:
        summonerId (str, optional): Summoner ID. Defaults to None.
        gameName (str, optional): Riot ID. Defaults to None.
        tagLine (str, optional): Riot Tag. Defaults to None.
        region (str, optional): Region. Defaults to 'americas'.
        
    Returns:
        str: puuid
    """
# using summonerID 
    if summonerId is not None:
        root_url = f'https://{region}.api.riotgames.com/'
        endpoint = 'lol/summoner/v4/summoners/'
        print(root_url+endpoint+summonerId+'?api_key='+api_key)
        response = requests.get(root_url+endpoint+summonerId+'?api_key='+api_key)

        return response.json()['puuid']
    else:
        #using gamename + tagline for example HGILLIS + NA1
        root_url = f'https://{region}.api.riotgames.com/'
        endpoint = f'riot/account/v1/accounts/by-riot-id/{gameName}/{tagLine}'

        response = requests.get(root_url+endpoint+'?api_key='+api_key)

        return response.json()['puuid']
    


## get gameName and tagLine from puuid

In [82]:

# this code will output like this: 'SummonerName#1234' for use in other functions


#use the players puuid to get their name and region code
def get_name_and_tag(puuid=None, api_key=None):
    link = f'https://americas.api.riotgames.com/riot/account/v1/accounts/by-puuid/{puuid}?api_key={api_key}'

    response = requests.get(link)

    gameName = response.json()['gameName']
    tagLine = response.json()['tagLine']

    return f'{gameName}#{tagLine}'

In [83]:
# this code will output like this: {'gameName': 'SummonerName', 'tagLine': '1234'} for further processing

def get_idtag_from_puuid(puuid=None):
    """Gets the riot_id and riot_tag from a puuid
    
    Args:
        puuid (str, optional): puuid. Defaults to None.
        
    Returns:
        id (dict): Dictionary with riot_id and riot_tag.
    """
    root_url = 'https://americas.api.riotgames.com/'
    endpoint = 'riot/account/v1/accounts/by-puuid/'

    response = requests.get(root_url+endpoint+puuid+'?api_key='+api_key)

    id = {
        'gameName': response.json()['gameName'],
        'tagLine': response.json()['tagLine']
    }

    return id

## retrive matches in match history using region and puuid. pick how many with start and count

In [84]:
def get_match_history(region=None, puuid=None, start=0, count=20):
    root_url = f'https://{region}.api.riotgames.com/'
    endpoint = f'lol/match/v5/matches/by-puuid/{puuid}/ids'
    query_params = f'?start={start}&count={count}' #changing starting index (start) and number of matches retrieved (count)

    response = requests.get(root_url + endpoint + query_params + '&api_key=' + api_key)
    
    return response.json()



## retrieve the data for an individual match

In [85]:
def get_match_data_from_id(region = None, matchId = None):

    root_url = f'https://{region}.api.riotgames.com/'
    endpoint = f'lol/match/v5/matches/{matchId}'


    response = requests.get(root_url + endpoint + '?api_key=' + api_key)

    return response.json()

## Take match info json and put it into a dataframe

In [86]:


def process_match_json(match_json, puuid):
    ## Architecture
    # left side is my name, right side is riots name
    metadata = match_json['metadata']
    info = match_json['info']
    game_mode = info['gameMode']
    players = info['participants']
    participants = metadata['participants']
    teams = info['teams']
    player = players[participants.index(puuid)]
    perks = player['perks']
    stats = perks ['statPerks']
    styles = perks['styles']

    primary = styles[0]
    secondary = styles[1]

    match_id = metadata['matchId']


    game_creation = info['gameCreation']
    game_duration = info['gameDuration']
    game_start_timestamp = info['gameStartTimestamp']
    game_end_timestamp = info['gameEndTimestamp']
    patch = info['gameVersion']

    riot_id = player['riotIdGameName']
    riot_tag = player['riotIdTagline']
    summoner_id = player['summonerId']
    summoner_name = player['summonerName']

    win = player['win']

    champ_id = player['championId']
    champ_transform = player['championTransform']
    champ_level = player['champLevel']

    gold_earned = player['goldEarned']
    neutral_minions_killed = player['neutralMinionsKilled']
    total_minions_killed = player['totalMinionsKilled']

    kills = player['kills']
    deaths = player['deaths']
    assists = player['assists']
    first_blood = player['firstBloodKill']

    total_damage_dealt= player['totalDamageDealtToChampions']
    total_damage_shielded = player['totalDamageShieldedOnTeammates']
    total_damage_taken = player['totalDamageTaken']
    total_damage_healed = player['totalHealsOnTeammates']
    total_minions_killed = player['totalMinionsKilled']
    total_time_cc_dealt = player['totalTimeCCDealt']

    early_surrender = player['gameEndedInEarlySurrender']
    surrender = player['gameEndedInSurrender']

    item0 = player['item0']
    item1 = player['item1']
    item2 = player['item2']
    item3 = player['item3']
    item4 = player['item4']
    item5 = player['item5']
    item6 = player['item6']

    summoner_1_id = player['summoner1Id']
    summoner_2_id = player['summoner2Id']


    wards_placed = player['wardsPlaced']
    vision_score = player['visionScore']







    defense = stats['defense']
    flex = stats['flex']
    offense = stats['offense']


    primary_style = primary['style']
    secondary_style = primary['style']

    primary_keystone = primary['selections'][0]['perk']
    primary_perk_1 = primary['selections'][1]['perk']
    primary_perk_2 = primary['selections'][2]['perk']
    primary_perk_3 = primary['selections'][3]['perk']

    secondary_perk_1 = secondary['selections'][0]['perk']
    secondary_perk_2 = secondary['selections'][1]['perk']


    objectives_stolen = player['objectivesStolen']
    objectives_stolen_assists = player['objectivesStolenAssists']

    ##what is this?
    detector_wars_placed = player['detectorWardsPlaced']
    vision_wards_bought = player['visionWardsBoughtInGame']
    role = player['role']

    damage_dealt_to_buildings = player['damageDealtToBuildings']
    damage_dealt_to_objectives = player['damageDealtToObjectives']
    damage_dealt_to_turrets = player['damageDealtToTurrets']
    game_ended_in_surrender = player['gameEndedInSurrender']
    gold_earned = player['goldEarned']
    individual_position = player['individualPosition']
    inhibitor_takedowns = player['inhibitorTakedowns']
    largest_killing_spree = player['largestKillingSpree']
    largest_multi_kill = player['largestMultiKill']
    total_damage_shielded_on_teammates = player['totalDamageShieldedOnTeammates']
    total_heals_on_teammates = player['totalHealsOnTeammates']
    total_minions_killed = player['totalMinionsKilled']
    vision_score = player['visionScore']


    matchDF = pd.DataFrame({
        'match_id': [match_id],
        'game_mode': [game_mode],
        'participants': [participants],
        'game_creation': [game_creation],
        'game_duration': [game_duration],
        'game_start_timestamp': [game_start_timestamp],
        'game_end_timestamp': [game_end_timestamp],
        'patch': [patch],
        'puuid': [puuid],
        'riot_id': [riot_id],
        'riot_tag': [riot_tag],
        'summoner_id': [summoner_id],
        'summoner_name': [summoner_name],
        'win': [win],
        'champion': [champ_id],
        'champion_transform': [champ_transform],
        'champion_level': [champ_level],
        'kills': [kills],
        'deaths': [deaths],
        'assists': [assists],
        'summoner1_id': [summoner_1_id],
        'summoner2_id': [summoner_2_id],
        'gold_earned': [gold_earned],
        'total_minions_killed': [total_minions_killed],
        'total_neutral_minions_killed': [neutral_minions_killed],
        'early_surrender': [early_surrender],
        'surrender': [surrender],
        'first_blood': [first_blood],
        'objectives_stolen': [objectives_stolen],
        'objectives_stolen_assists': [objectives_stolen_assists],
        'total_damage_dealt_champions': [total_damage_dealt],
        'total_damage_taken': [total_damage_taken],
        'total_damage_shielded_teammates': [total_damage_shielded],
        'total_time_crowd_controlled': [total_time_cc_dealt],
        'vision_score': [vision_score],
        'wards_placed': [wards_placed],
        'item0': [item0],
        'item1': [item1],
        'item2': [item2],
        'item3': [item3],
        'item4': [item4],
        'item5': [item5],
        'item6': [item6],
        'perk_keystone': [primary_keystone],
        'perk_primary_row_1': [primary_perk_1],
        'perk_primary_row_2': [primary_perk_2],
        'perk_primary_row_3': [primary_perk_3],
        'perk_secondary_row_1': [secondary_perk_1],
        'perk_secondary_row_2': [secondary_perk_2],
        'perk_primary_style': [primary_style],
        'perk_secondary_style': [secondary_style],
        'perk_shard_defense': [defense],
        'perk_shard_flex': [flex],
        'perk_shard_offense': [offense],
        'damage_dealt_to_buildings': [damage_dealt_to_buildings],
        'damage_dealt_to_objectives': [damage_dealt_to_objectives],
        'damage_dealt_to_turrets': [damage_dealt_to_turrets],
        'game_ended_in_surrender': [game_ended_in_surrender],
        'individual_position': [individual_position],
        'inhibitor_takedowns': [inhibitor_takedowns],
        'largest_killing_spree': [largest_killing_spree],
        'largest_multi_kill': [largest_multi_kill],
        'total_damage_shielded_on_teammates': [total_damage_shielded_on_teammates],
        'total_heals_on_teammates': [total_heals_on_teammates],
        'total_minions_killed': [total_minions_killed],
        'vision_score': [vision_score],
    })


    return matchDF


## use functions

In [87]:
#test get puuid function
get_puuid(gameName=gameName, tagLine=tagLine)

'V8lkRF0RP5WLyEAM7kf0F4RJanDfdGveBZESxJBZPxaRoDgxwUtdIeVTZrmOzjeVHklBRlCJr1tz1g'

In [88]:
#test id function
get_idtag_from_puuid(get_puuid(gameName=gameName, tagLine=tagLine))

{'gameName': 'HGILLIS', 'tagLine': 'NA1'}

In [89]:
#test get_match_history
match_history = get_match_history(region=region, puuid=get_puuid(gameName=gameName, tagLine=tagLine), start=0, count=5)

In [90]:
#test match data function. Use this url, and the middle option to understand output: https://developer.riotgames.com/apis#match-v5/GET_getMatch
game = get_match_data_from_id(region='americas', matchId=match_history[0])

game

{'metadata': {'dataVersion': '2',
  'matchId': 'NA1_5183290141',
  'participants': ['d66zRs2wr3In3cbz_fPo1cbUt99SyQNUr78FGsGcm_BmaZNY9puvPrt_j6b7zXv5pXNg4qoBETm5aQ',
   '-qcVfkj8M4tsiLxecUkLkTk8OtK7OBZwvihx3LOKlFuJY0svnxxBSyZiYVVCkwL4hwWl1CJuSOFA7w',
   'v9cKAiyHdtPQ4HXfm7RKoUiH4Qme9tpon2h-j6977gySZVShcbpHA_rqcGRtvR_jdIltku3KSZRBYw',
   'LOSW_2HtLudq3IO26xWlzSEQr_YBdH2fIsP0u3fhXG49qecrSx-uYSv_vq59IbyHv_GZ06O7RZBfaA',
   'V8lkRF0RP5WLyEAM7kf0F4RJanDfdGveBZESxJBZPxaRoDgxwUtdIeVTZrmOzjeVHklBRlCJr1tz1g',
   'SorCrpsgNFXHdNcdZ2m47enpTSV43CwGc7rhYtLS6JpRXUNkpXnsH7XyliWVPAQQCiFw2_Y0FTgyBQ',
   'an22PM83C73bkVp1VYrOn9bDx9aLW2hM2hVOzaEw_IBrTFlkHoNH866iPWTrqlhG1WvpQrnaY2DHmg',
   '2Jmj26CzoRJm43oNW2HK9CB_Ucl39dragYHTE0gQMeF1FbM1S-867TT3YE5dQ5itjHJhOegHQTHmlw',
   '6bDhHEQj8Dn3MAFoMG98jvGVEZ18n0jE2w4nJzAr5ijmYoyivySMNG_DDvRseIQRCMMGrdHRzqER7g',
   'pLsWfTCfaiBpcgQig8hDvRvGBkU2QkOfoop8lxtHBCqo1fAE4uC83XKA_iPwitGa0oHFwPbbUDsqVA']},
 'info': {'endOfGameResult': 'GameComplete',
  'gameCreation': 1734

In [91]:
# test processing a single game
process_match_json(game, get_puuid(gameName=gameName, tagLine=tagLine))

,match_id,game_mode,participants,game_creation,game_duration,game_start_timestamp,game_end_timestamp,patch,puuid,riot_id,riot_tag,summoner_id,summoner_name,win,champion,champion_transform,champion_level,kills,deaths,assists,summoner1_id,summoner2_id,gold_earned,total_minions_killed,total_neutral_minions_killed,early_surrender,surrender,first_blood,objectives_stolen,objectives_stolen_assists,total_damage_dealt_champions,total_damage_taken,total_damage_shielded_teammates,total_time_crowd_controlled,vision_score,wards_placed,item0,item1,item2,item3,item4,item5,item6,perk_keystone,perk_primary_row_1,perk_primary_row_2,perk_primary_row_3,perk_secondary_row_1,perk_secondary_row_2,perk_primary_style,perk_secondary_style,perk_shard_defense,perk_shard_flex,perk_shard_offense,damage_dealt_to_buildings,damage_dealt_to_objectives,damage_dealt_to_turrets,game_ended_in_surrender,individual_position,inhibitor_takedowns,largest_killing_spree,largest_multi_kill,total_damage_shielded_on_teammates,total_heals_on_teammates
0,NA1_5183290141,CLASSIC,[d66zRs2wr3In3cbz_fPo1cbUt99SyQNUr78FGsGcm_Bma...,1734323501921,2187,1734323746573,1734325933833,14.24.643.5128,V8lkRF0RP5WLyEAM7kf0F4RJanDfdGveBZESxJBZPxaRoD...,HGILLIS,NA1,yw8eql-EBvf1djPzM__tH1wGBOqqiPaYbtHx3O6pBOO7bZI,HGILLIS,True,26,0,16,1,4,29,7,4,10596,24,0,False,False,False,0,0,9960,32102,1835,181,45,21,3876,2065,3158,4005,3083,3113,3364,8214,8226,8210,8232,8126,8105,8200,8200,5001,5008,5007,2034,2135,2034,False,UTILITY,0,0,1,1835,2550


In [92]:
# get match ids using match history
match_ids = get_match_history(region='americas', puuid=get_puuid(gameName=gameName, tagLine=tagLine))

# fill a df with each match id, then use the get match data and process it into the other columns
df = pd.DataFrame()
for match_id in match_ids:
    game = get_match_data_from_id(region='americas', matchId=match_id)
    matchDF = process_match_json(game, puuid=get_puuid(gameName=gameName, tagLine=tagLine))

    df = pd.concat([df, matchDF])

In [93]:
df

,match_id,game_mode,participants,game_creation,game_duration,game_start_timestamp,game_end_timestamp,patch,puuid,riot_id,riot_tag,summoner_id,summoner_name,win,champion,champion_transform,champion_level,kills,deaths,assists,summoner1_id,summoner2_id,gold_earned,total_minions_killed,total_neutral_minions_killed,early_surrender,surrender,first_blood,objectives_stolen,objectives_stolen_assists,total_damage_dealt_champions,total_damage_taken,total_damage_shielded_teammates,total_time_crowd_controlled,vision_score,wards_placed,item0,item1,item2,item3,item4,item5,item6,perk_keystone,perk_primary_row_1,perk_primary_row_2,perk_primary_row_3,perk_secondary_row_1,perk_secondary_row_2,perk_primary_style,perk_secondary_style,perk_shard_defense,perk_shard_flex,perk_shard_offense,damage_dealt_to_buildings,damage_dealt_to_objectives,damage_dealt_to_turrets,game_ended_in_surrender,individual_position,inhibitor_takedowns,largest_killing_spree,largest_multi_kill,total_damage_shielded_on_teammates,total_heals_on_teammates
0,NA1_5183290141,CLASSIC,[d66zRs2wr3In3cbz_fPo1cbUt99SyQNUr78FGsGcm_Bma...,1734323501921,2187,1734323746573,1734325933833,14.24.643.5128,V8lkRF0RP5WLyEAM7kf0F4RJanDfdGveBZESxJBZPxaRoD...,HGILLIS,NA1,yw8eql-EBvf1djPzM__tH1wGBOqqiPaYbtHx3O6pBOO7bZI,HGILLIS,True,26,0,16,1,4,29,7,4,10596,24,0,False,False,False,0,0,9960,32102,1835,181,45,21,3876,2065,3158,4005,3083,3113,3364,8214,8226,8210,8232,8126,8105,8200,8200,5001,5008,5007,2034,2135,2034,False,UTILITY,0,0,1,1835,2550
0,NA1_5183249627,CLASSIC,[d66zRs2wr3In3cbz_fPo1cbUt99SyQNUr78FGsGcm_Bma...,1734321321424,1858,1734321401751,1734323259816,14.24.643.5128,V8lkRF0RP5WLyEAM7kf0F4RJanDfdGveBZESxJBZPxaRoD...,HGILLIS,NA1,yw8eql-EBvf1djPzM__tH1wGBOqqiPaYbtHx3O6pBOO7bZI,HGILLIS,True,142,0,13,4,10,11,14,4,8870,10,0,False,True,False,0,0,17971,21762,0,135,39,17,3871,3020,6655,4628,3057,1052,3364,8128,8143,8138,8135,8233,8236,8100,8100,5001,5008,5008,1749,2599,1749,True,UTILITY,0,2,2,0,164
0,NA1_5181444806,CLASSIC,[zidX4wyMk3i2Kogfu0UrrxWhbwAgQ1nBHvCwdEi287nAS...,1734153465699,1733,1734153550843,1734155283741,14.24.643.5128,V8lkRF0RP5WLyEAM7kf0F4RJanDfdGveBZESxJBZPxaRoD...,HGILLIS,NA1,yw8eql-EBvf1djPzM__tH1wGBOqqiPaYbtHx3O6pBOO7bZI,HGILLIS,False,80,0,12,5,8,11,14,4,8986,20,0,False,False,False,0,0,13435,20347,0,39,54,21,3877,3142,3013,3179,3134,1037,3364,8005,9111,9105,8014,8126,8105,8000,8000,5001,5008,5008,258,3157,258,False,UTILITY,0,0,1,0,0
0,NA1_5181412050,CLASSIC,[d66zRs2wr3In3cbz_fPo1cbUt99SyQNUr78FGsGcm_Bma...,1734151584129,1476,1734151657800,1734153134085,14.24.643.5128,V8lkRF0RP5WLyEAM7kf0F4RJanDfdGveBZESxJBZPxaRoD...,HGILLIS,NA1,yw8eql-EBvf1djPzM__tH1wGBOqqiPaYbtHx3O6pBOO7bZI,HGILLIS,True,427,0,13,3,3,25,7,4,8573,8,0,False,False,False,0,0,7379,12907,9572,109,24,9,3870,3158,6617,3504,3113,4642,3364,8214,8226,8210,8237,8463,8453,8200,8200,5001,5008,5007,3937,4101,3937,False,UTILITY,2,2,1,9572,1343
0,NA1_5179458819,CLASSIC,[CBD1l26RzaFFa-cImBA7O9wwGvH3h7h3vxTCfqbY8UbgU...,1733964142972,3042,1733964228361,1733967270880,14.24.642.1879,V8lkRF0RP5WLyEAM7kf0F4RJanDfdGveBZESxJBZPxaRoD...,HGILLIS,NA1,yw8eql-EBvf1djPzM__tH1wGBOqqiPaYbtHx3O6pBOO7bZI,HGILLIS,True,27,0,18,3,18,23,6,4,12787,22,2,False,False,False,0,0,16730,62896,0,334,83,33,3876,2065,3116,3002,3013,1011,3364,8360,8306,8345,8410,5007,5010,8300,8300,5011,5010,5007,724,3594,724,False,UTILITY,0,0,1,0,0
0,NA1_5179431661,CLASSIC,[-34neXe-6Jv5gh-aG1oG30Qj4K2_nUyaQk7aKYAdmlP75...,1733962492140,1296,1733962606178,1733963902677,14.24.642.1879,V8lkRF0RP5WLyEAM7kf0F4RJanDfdGveBZESxJBZPxaRoD...,HGILLIS,NA1,yw8eql-EBvf1djPzM__tH1wGBOqqiPaYbtHx3O6pBOO7bZI,HGILLIS,False,99,0,10,2,7,4,14,4,5653,23,0,False,True,False,0,0,10801,11387,839,134,30,11,3871,3020,3118,1052,0,0,3364,8128,8126,8138,8106,5008,5001,8100,8100,5008,5001,5008,247,467,247,True,UTILITY,0,0,1,839,0
0,NA1_5176298198,CLASSIC,[-qcVfkj8M4tsiLxecUkLkTk8OtK7OBZwvihx3LOKlFuJY...,1733630947328,1593,1733630972745,1733632566345,14.23.636.9832,V8lkRF0RP5WLyEAM7kf0F4RJanDfdGveBZESxJBZPxaRoD..

# Community Dragon

In [94]:
# keys to go from numbers to actual names

perk = 'https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/perks.json'
perk_styles = 'https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/perkstyles.json'
champ = 'https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/champion-summary.json'
item = 'https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/items.json'

perk_json = requests.get(perk).json()
perk_styles_json = requests.get(perk_styles).json()
champ_json = requests.get(champ).json()
item_json = requests.get(item).json()

In [95]:

def json_extract(obj, key):
    arr = []
    
    def extract(obj, arr, key):
        if isinstance(obj, dict):
            for k, v in obj.items():
                if k == key:
                    arr.append(v)
                elif isinstance(v, (dict, list)):
                    extract(v, arr, key)
        elif isinstance(obj, list):
            for item in obj:
                extract(item, arr, key)
        return arr

    values = extract(obj, arr, key)
    return values


In [96]:
# Base URL for the champion icons
base_url = 'https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/champion-icons/'

# Add a new column with the image URLs
df['image_url'] = df['champion'].apply(lambda x: f"{base_url}{x}.png")

# Display the DataFrame
print(df)


         match_id game_mode  \
0  NA1_5183290141   CLASSIC   
0  NA1_5183249627   CLASSIC   
0  NA1_5181444806   CLASSIC   
0  NA1_5181412050   CLASSIC   
0  NA1_5179458819   CLASSIC   
0  NA1_5179431661   CLASSIC   
0  NA1_5176298198   CLASSIC   
0  NA1_5175485627   CLASSIC   
0  NA1_5175446757   CLASSIC   
0  NA1_5175002848      ARAM   
0  NA1_5174474532   CLASSIC   
0  NA1_5174425187   CLASSIC   
0  NA1_5173573762      ARAM   
0  NA1_5173530695      ARAM   
0  NA1_5173479697   CLASSIC   
0  NA1_5173454760   CLASSIC   
0  NA1_5173410483   CLASSIC   
0  NA1_5173401158   CLASSIC   
0  NA1_5173362198   CLASSIC   
0  NA1_5173330235   CLASSIC   

                                        participants  game_creation  \
0  [d66zRs2wr3In3cbz_fPo1cbUt99SyQNUr78FGsGcm_Bma...  1734323501921   
0  [d66zRs2wr3In3cbz_fPo1cbUt99SyQNUr78FGsGcm_Bma...  1734321321424   
0  [zidX4wyMk3i2Kogfu0UrrxWhbwAgQ1nBHvCwdEi287nAS...  1734153465699   
0  [d66zRs2wr3In3cbz_fPo1cbUt99SyQNUr78FGsGcm_Bma...  1734151584

In [97]:
from IPython.display import display, Image

# Display all champion images from the DataFrame
for image_url in df['image_url']:
    display(Image(url=image_url))


In [98]:
# code to replace all of the perk numbers with the actual perks

perk_ids = json_extract(perk_json, 'id')
perk_names = json_extract(perk_json, 'name')

perk_dict = dict(map(lambda i, j : (int(i),j), perk_ids, perk_names))

perk_styles_ids = json_extract(perk_styles_json, 'id')
perk_styles_names = json_extract(perk_styles_json, 'name')

perk_styles_dict = dict(map(lambda i, j : (int(i),j), perk_styles_ids, perk_styles_names))

In [99]:
# code to replace all of the champion numbers with the actual champions
champ_ids = json_extract(champ_json, 'id')
champ_names = json_extract(champ_json, 'name')

# Assuming champ_json is a list of dictionaries with each dictionary containing 'id' and 'name'
champ_dict = {int(champ['id']): champ['name'] for champ in champ_json}



In [100]:
# code to replace all of the item numbers with the actual items
item_ids = json_extract(item_json, 'id')
item_names = json_extract(item_json, 'name')

# Assuming champ_json is a list of dictionaries with each dictionary containing 'id' and 'name'
item_dict = {int(item['id']): item['name'] for item in item_json}


In [101]:
pd.options.display.max_columns = 100

df.replace(perk_dict).replace(perk_styles_dict)

# Replace only the 'champion' column with the values from champ_dict
df['champion'] = df['champion'].replace(champ_dict)

# replace the item columns with the the actual items names from the item_dict
df['item0'] = df['item0'].replace(item_dict)
df['item1'] = df['item1'].replace(item_dict)
df['item2'] = df['item2'].replace(item_dict)
df['item3'] = df['item3'].replace(item_dict)
df['item4'] = df['item4'].replace(item_dict)
df['item5'] = df['item5'].replace(item_dict)
df['item6'] = df['item6'].replace(item_dict)
df


,match_id,game_mode,participants,game_creation,game_duration,game_start_timestamp,game_end_timestamp,patch,puuid,riot_id,riot_tag,summoner_id,summoner_name,win,champion,champion_transform,champion_level,kills,deaths,assists,summoner1_id,summoner2_id,gold_earned,total_minions_killed,total_neutral_minions_killed,early_surrender,surrender,first_blood,objectives_stolen,objectives_stolen_assists,total_damage_dealt_champions,total_damage_taken,total_damage_shielded_teammates,total_time_crowd_controlled,vision_score,wards_placed,item0,item1,item2,item3,item4,item5,item6,perk_keystone,perk_primary_row_1,perk_primary_row_2,perk_primary_row_3,perk_secondary_row_1,perk_secondary_row_2,perk_primary_style,perk_secondary_style,perk_shard_defense,perk_shard_flex,perk_shard_offense,damage_dealt_to_buildings,damage_dealt_to_objectives,damage_dealt_to_turrets,game_ended_in_surrender,individual_position,inhibitor_takedowns,largest_killing_spree,largest_multi_kill,total_damage_shielded_on_teammates,total_heals_on_teammates,image_url
0,NA1_5183290141,CLASSIC,[d66zRs2wr3In3cbz_fPo1cbUt99SyQNUr78FGsGcm_Bma...,1734323501921,2187,1734323746573,1734325933833,14.24.643.5128,V8lkRF0RP5WLyEAM7kf0F4RJanDfdGveBZESxJBZPxaRoD...,HGILLIS,NA1,yw8eql-EBvf1djPzM__tH1wGBOqqiPaYbtHx3O6pBOO7bZI,HGILLIS,True,Zilean,0,16,1,4,29,7,4,10596,24,0,False,False,False,0,0,9960,32102,1835,181,45,21,Solstice Sleigh,Shurelya's Battlesong,Ionian Boots of Lucidity,Imperial Mandate,Warmog's Armor,Aether Wisp,Oracle Lens,8214,8226,8210,8232,8126,8105,8200,8200,5001,5008,5007,2034,2135,2034,False,UTILITY,0,0,1,1835,2550,https://raw.communitydragon.org/latest/plugins...
0,NA1_5183249627,CLASSIC,[d66zRs2wr3In3cbz_fPo1cbUt99SyQNUr78FGsGcm_Bma...,1734321321424,1858,1734321401751,1734323259816,14.24.643.5128,V8lkRF0RP5WLyEAM7kf0F4RJanDfdGveBZESxJBZPxaRoD...,HGILLIS,NA1,yw8eql-EBvf1djPzM__tH1wGBOqqiPaYbtHx3O6pBOO7bZI,HGILLIS,True,Zoe,0,13,4,10,11,14,4,8870,10,0,False,True,False,0,0,17971,21762,0,135,39,17,Zaz'Zak's Realmspike,Sorcerer's Shoes,Luden's Companion,Horizon Focus,Sheen,Amplifying Tome,Oracle Lens,8128,8143,8138,8135,8233,8236,8100,8100,5001,5008,5008,1749,2599,1749,True,UTILITY,0,2,2,0,164,https://raw.communitydragon.org/latest/plugins...
0,NA1_5181444806,CLASSIC,[zidX4wyMk3i2Kogfu0UrrxWhbwAgQ1nBHvCwdEi287nAS...,1734153465699,1733,1734153550843,1734155283741,14.24.643.5128,V8lkRF0RP5WLyEAM7kf0F4RJanDfdGveBZESxJBZPxaRoD...,HGILLIS,NA1,yw8eql-EBvf1djPzM__tH1wGBOqqiPaYbtHx3O6pBOO7bZI,HGILLIS,False,Pantheon,0,12,5,8,11,14,4,8986,20,0,False,False,False,0,0,13435,20347,0,39,54,21,Bloodsong,Youmuu's Ghostblade,Synchronized Souls,Umbral Glaive,Serrated Dirk,Pickaxe,Oracle Lens,8005,9111,9105,8014,8126,8105,8000,8000,5001,5008,5008,258,3157,258,False,UTILITY,0,0,1,0,0,https://raw.communitydragon.org/latest/plugins...
0,NA1_5181412050,CLASSIC,[d66zRs2wr3In3cbz_fPo1cbUt99SyQNUr78FGsGcm_Bma...,1734151584129,1476,1734151657800,1734153134085,14.24.643.5128,V8lkRF0RP5WLyEAM7kf0F4RJanDfdGveBZESxJBZPxaRoD...,HGILLIS,NA1,yw8eql-EBvf1djPzM__tH1wGBOqqiPaYbtHx3O6pBOO7bZI,HGILLIS,True,Ivern,0,13,3,3,25,7,4,8573,8,0,False,False,False,0,0,7379,12907,9572,109,24,9,Dream Maker,Ionian Boots of Lucidity,Moonstone Renewer,Ardent Censer,Aether Wisp,Bandleglass Mirror,Oracle Lens,8214,8226,8210,8237,8463,8453,8200,8200,5001,5008,5007,3937,4101,3937,False,UTILITY,2,2,1,9572,1343,https://raw.communitydragon.org/latest/plugins...
0,NA1_5179458819,CLASSIC,[CBD1l26RzaFFa-cImBA7O9wwGvH3h7h3vxTCfqbY8UbgU...,1733964142972,3042,1733964228361,1733967270880,14.24.642.1879,V8lkRF0RP5WLyEAM7kf0F4RJanDfdGveBZESxJBZPxaRoD...,HGILLIS,NA1,yw8eql-EBvf1djPzM__tH1wGBOqqiPaYbtHx3O6pBOO7bZI,HGILLIS,True,Singed,0,18,3,18,23,6,4,12787,22,2,False,False,False,0,0,16730,62896,0,334,83,33,Solstice Sleigh,Shurelya's Battlesong,Rylai's Crystal Scepter,Trailblazer,Synchronized Souls,Giant's Belt,Oracle Lens,8360,8306,8345,8410,5007,5010,8300,8300,5011,5010,5007,724,3594,724,False,UTILITY,0,0,1,0,0,https://raw.communitydragon.org/latest/plu

In [103]:
# Replace the values based on the dictionaries and drop the first 12 columns
filtered_df = df.replace(perk_dict).replace(perk_styles_dict).iloc[:10, 12:]

#drop columns i dont want
filtered_df = filtered_df.drop(columns = ['champion_transform', 'total_neutral_minions_killed', 'early_surrender', 'surrender',  'first_blood', 'objectives_stolen',  'objectives_stolen_assists' ])

# Replace True with 'Win' and False with 'Loss' using map
# Replace True with 'Win' and False with 'Loss'
filtered_df['win'] = filtered_df['win'].replace({True: 'Win', False: 'Loss'})




# Output the result to a CSV file
filtered_df.to_csv('match_histAPI.csv', index=False)

filtered_df


,summoner_name,win,champion,champion_level,kills,deaths,assists,summoner1_id,summoner2_id,gold_earned,total_minions_killed,total_damage_dealt_champions,total_damage_taken,total_damage_shielded_teammates,total_time_crowd_controlled,vision_score,wards_placed,item0,item1,item2,item3,item4,item5,item6,perk_keystone,perk_primary_row_1,perk_primary_row_2,perk_primary_row_3,perk_secondary_row_1,perk_secondary_row_2,perk_primary_style,perk_secondary_style,perk_shard_defense,perk_shard_flex,perk_shard_offense,damage_dealt_to_buildings,damage_dealt_to_objectives,damage_dealt_to_turrets,game_ended_in_surrender,individual_position,inhibitor_takedowns,largest_killing_spree,largest_multi_kill,total_damage_shielded_on_teammates,total_heals_on_teammates,image_url
0,HGILLIS,Win,Zilean,16,1,4,29,7,4,10596,24,9960,32102,1835,181,45,21,Solstice Sleigh,Shurelya's Battlesong,Ionian Boots of Lucidity,Imperial Mandate,Warmog's Armor,Aether Wisp,Oracle Lens,Summon Aery,Manaflow Band,Transcendence,Waterwalking,Cheap Shot,Relentless Hunter,Sorcery,Sorcery,Health Scaling,Adaptive Force,Ability Haste,2034,2135,2034,False,UTILITY,0,0,1,1835,2550,https://raw.communitydragon.org/latest/plugins...
0,HGILLIS,Win,Zoe,13,4,10,11,14,4,8870,10,17971,21762,0,135,39,17,Zaz'Zak's Realmspike,Sorcerer's Shoes,Luden's Companion,Horizon Focus,Sheen,Amplifying Tome,Oracle Lens,Dark Harvest,Sudden Impact,Eyeball Collection,Treasure Hunter,Absolute Focus,Gathering Storm,Precision,Precision,Health Scaling,Adaptive Force,Adaptive Force,1749,2599,1749,True,UTILITY,0,2,2,0,164,https://raw.communitydragon.org/latest/plugins...
0,HGILLIS,Loss,Pantheon,12,5,8,11,14,4,8986,20,13435,20347,0,39,54,21,Bloodsong,Youmuu's Ghostblade,Synchronized Souls,Umbral Glaive,Serrated Dirk,Pickaxe,Oracle Lens,Press the Attack,Triumph,Legend: Haste,Coup de Grace,Cheap Shot,Relentless Hunter,Domination,Domination,Health Scaling,Adaptive Force,Adaptive Force,258,3157,258,False,UTILITY,0,0,1,0,0,https://raw.communitydragon.org/latest/plugins...
0,HGILLIS,Win,Ivern,13,3,3,25,7,4,8573,8,7379,12907,9572,109,24,9,Dream Maker,Ionian Boots of Lucidity,Moonstone Renewer,Ardent Censer,Aether Wisp,Bandleglass Mirror,Oracle Lens,Summon Aery,Manaflow Band,Transcendence,Scorch,Font of Life,Revitalize,Sorcery,Sorcery,Health Scaling,Adaptive Force,Ability Haste,3937,4101,3937,False,UTILITY,2,2,1,9572,1343,https://raw.communitydragon.org/latest/plugins...
0,HGILLIS,Win,Singed,18,3,18,23,6,4,12787,22,16730,62896,0,334,83,33,Solstice Sleigh,Shurelya's Battlesong,Rylai's Crystal Scepter,Trailblazer,Synchronized Souls,Giant's Belt,Oracle Lens,Unsealed Spellbook,Hextech Flashtraption,Biscuit Delivery,Approach Velocity,Ability Haste,Move Speed,Inspiration,Inspiration,Health,Move Speed,Ability Haste,724,3594,724,False,UTILITY,0,0,1,0,0,https://raw.communitydragon.org/latest/plugins...
0,HGILLIS,Loss,Lux,10,2,7,4,14,4,5653,23,10801,11387,839,134,30,11,Zaz'Zak's Realmspike,Sorcerer's Shoes,Malignance,Amplifying Tome,0,0,Oracle Lens,Dark Harvest,Cheap Shot,Eyeball Collection,Ultimate Hunter,Adaptive Force,Health Scaling,Precision,Precision,Adaptive Force,Health Scaling,Adaptive Force,247,467,247,True,UTILITY,0,0,1,839,0,https://raw.communitydragon.org/latest/plugins...
0,HGILLIS,Win,Graves,14,3,9,10,11,4,9643,11,13478,23269,0,409,14,6,The Collector,Youmuu's Ghostblade,Plated Steelcaps,B. F. Sword,0,0,Stealth Ward,Dark Harvest,Sudden Impact,Eyeball Collection,Treasure Hunter,Legend: Alacrity,Triumph,Precision,Precision,Health Scaling,Adaptive Force,Attack Speed,182,10975,182,False,JUNGLE,0,0,1,0,0,https://raw.communitydragon.org/latest/plugins...
0,HGILLIS,Win,Nocturne,14,9,4,11,11,4,12461,23,22108,29910,0,607,18,8,Experimental Hexplate,Stridebreaker,Plated Steelcaps,Eclipse,Pickaxe,0,Stealth Ward,Conqueror,Triumph,Legend: Alacrity,Last Stand,Eyeball Collection,Ultimate Hunter,Domination,Domination,Health Scaling,Adaptive Force,Attack Speed,5686,23557,5686,False,JUNGLE,1,3,2,0,0,https://raw.communitydragon.org/latest/plugins..